In [68]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS backend for training.")
else:
    device = torch.device("cpu")
    print("MPS backend not available, using CPU.")

class LRN(nn.Module):
    def __init__(self, size=5, alpha=1e-4, beta=0.75, k=2.0):
        super(LRN, self).__init__()
        self.size = size
        self.alpha = alpha
        self.beta = beta
        self.k = k

    def forward(self, x):
        div = x.pow(2)
        div = nn.functional.avg_pool2d(div, (self.size, self.size), stride=1, padding=(self.size // 2))
        div = div.mul(self.alpha).add(self.k).pow(self.beta)
        x = x / div
        return x

class CIFAR10AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(CIFAR10AlexNet, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            LRN(),  
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),  
            
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            LRN(),  
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),  

            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 384, kernel_size=3, padding=1),  
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1), 
        )

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),

            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),

            nn.Linear(4096, 1024),  
            nn.ReLU(inplace=True),

            nn.Linear(1024, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        if device == torch.device("mps"):  # If running on MPS, move to CPU for this operation
            x = x.cpu()
            x = self.avgpool(x)
            x = x.to(device)
        else:
            x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Instantiate the updated model
model = CIFAR10AlexNet(num_classes=10).to(device)


Using MPS backend for training.


PREPARE DATASET

In [69]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),             
    transforms.RandomHorizontalFlip(),                
    transforms.RandomRotation(15),                    
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  
    transforms.ToTensor(),                              
    transforms.Normalize((0.4914, 0.4822, 0.4465),      
                         (0.2023, 0.1994, 0.2010)),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value="random"),  
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


Define the Loss Function and Optimizer

In [70]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)


Train the Model on CIFAR-10

In [71]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

epochs = 10  

for epoch in range(epochs):
    model.train()  
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 100 == 99:
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print("Training complete")


Epoch 1, Batch 100, Loss: 2.303
Epoch 1, Batch 200, Loss: 2.303
Epoch 1, Batch 300, Loss: 2.303
Epoch 2, Batch 100, Loss: 2.303
Epoch 2, Batch 200, Loss: 2.303
Epoch 2, Batch 300, Loss: 2.303
Epoch 3, Batch 100, Loss: 2.303
Epoch 3, Batch 200, Loss: 2.302
Epoch 3, Batch 300, Loss: 2.302
Epoch 4, Batch 100, Loss: 2.280
Epoch 4, Batch 200, Loss: 2.150
Epoch 4, Batch 300, Loss: 2.085
Epoch 5, Batch 100, Loss: 2.015
Epoch 5, Batch 200, Loss: 1.978
Epoch 5, Batch 300, Loss: 1.970
Epoch 6, Batch 100, Loss: 1.934
Epoch 6, Batch 200, Loss: 1.922
Epoch 6, Batch 300, Loss: 1.895
Epoch 7, Batch 100, Loss: 1.840
Epoch 7, Batch 200, Loss: 1.809
Epoch 7, Batch 300, Loss: 1.777
Epoch 8, Batch 100, Loss: 1.723
Epoch 8, Batch 200, Loss: 1.684
Epoch 8, Batch 300, Loss: 1.669
Epoch 9, Batch 100, Loss: 1.638
Epoch 9, Batch 200, Loss: 1.602
Epoch 9, Batch 300, Loss: 1.594
Epoch 10, Batch 100, Loss: 1.563
Epoch 10, Batch 200, Loss: 1.528
Epoch 10, Batch 300, Loss: 1.505
Training complete


Evaluate the Model on CIFAR-10

In [72]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on the CIFAR-10 test set: {100 * correct / total:.2f}%')


Accuracy on the CIFAR-10 test set: 45.85%


SAVE THE MODEL

In [73]:
torch.save(model.state_dict(), 'CIFAR10_alexnet.pth')
